In [1]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
#Define presto credentials
presto_host = 'presto.processing.yoda.run'
presto_port = '80'
username = 'aditya.bhattar@rapido.bike'

#Create connection to presto host
connection = presto.connect(presto_host,presto_port,username = username)

In [12]:
#Query data from presto
q = """
select * from datasets.composite_hex_config
"""

#Load data into pandas table
df_copy = pd.read_sql(q, connection)

In [13]:
#Make a copy of the df
df = df_copy.copy()
df.head()

,composite_hex_id,source_hex,time_of_day,day_of_week,city_id,refresh_date,start_time,end_time,service_detail_id,min_supply_per_order,batch_window_in_secs,priority_service_detail,priority_service_category,priority_attribute,enabled,city_name
0,ch0000000294,883ca8526bfffff,afternoon,Monday,5bc5afee12477c2ece7695de,20220117,12:00:00,17:00:00,5c7594734666ec5e4fe61294,1,20,"5c7594734666ec5e4fe61294,60c1dee3b8fa2a234b51e...",B2C,service_detail,True,ranchi
1,ch0000000294,883ca8526bfffff,evening_peak,Monday,5bc5afee12477c2ece7695de,20220117,17:00:00,22:00:00,5c7594734666ec5e4fe61294,1,15,"5c7594734666ec5e4fe61294,60c1dee3b8fa2a234b51e...",B2C,service_detail,True,ranchi
2,ch0000000294,883ca8526bfffff,evening_peak,Monday,5bc5afee12477c2ece7695de,20220117,17:00:00,22:00:00,5df237549ac51d6c6484c0d6,2,15,"5c7594734666ec5e4fe61294,60c1dee3b8fa2a234b51e...",B2C,service_detail,True,ranchi
3,ch0000000317,883da2199bfffff,afternoon,Monday,5ba090686fde19440c388a07,20220117,12:00:00,17:00:00,5ba1f6d0be5b826084cc7202,1,20,"5ba1f6d0be5b826084cc7202,5c7a573f19be406b18c49...",B2C,service_detail,False,jaipur
4,ch0000000294,883ca8526bfffff,evening_peak,Tuesday,5bc5afee12477c2ece7695de,20220117,17:00:00,22:00:00,5c7594734666ec5e4fe61294,1,15,"5c7594734666ec5e4fe61294,60c1dee3b8fa2a234b51e...",B2C,service_detail,True,ranchi


In [15]:
df[df['city_name'] == 'hyderabad']['composite_hex_id'].unique()

array(['ch000000398'], dtype=object)

In [5]:
#Filter df on enabled == true
df_filter = df[df['enabled'] == True]
df_filter.head()

,composite_hex_id,source_hex,time_of_day,day_of_week,city_id,refresh_date,start_time,end_time,service_detail_id,min_supply_per_order,batch_window_in_secs,priority_service_detail,priority_service_category,priority_attribute,enabled,city_name


In [6]:
#Get list of the hexes where gp is enabled
gp_enabled_hexes = list(df_filter['source_hex'].unique())

In [19]:
#Load the hexes identified for all the cities

#Bangalore
bangalore_area1 = pd.read_csv('/Users/rapido/Desktop/batching_demand/bangalore_location_data_level1_demand_zone1.csv')
bangalore_area1_hexes = list(bangalore_area1['customer_location_hex_8'].unique())
bangalore_area2 = pd.read_csv('/Users/rapido/Desktop/batching_demand/bangalore_location_data_level1_demand_zone2.csv')
bangalore_area2_hexes = list(bangalore_area2['customer_location_hex_8'].unique())

#Hyderabad
hyderabad_area1 = pd.read_csv('/Users/rapido/Desktop/batching_demand/hyderabad/hyderabad_location_data_level1_demand_zone1.csv')
hyderabad_area1_hexes = list(hyderabad_area1['customer_location_hex_8'].unique())
hyderabad_area2 = pd.read_csv('/Users/rapido/Desktop/batching_demand/hyderabad/hyderabad_location_data_level1_demand_zone2.csv')
hyderabad_area2_hexes = list(hyderabad_area2['customer_location_hex_8'].unique())

#Delhi
delhi_area1 = pd.read_csv('/Users/rapido/Desktop/batching_demand/delhi/delhi_location_data_level1_demand_zone1.csv')
delhi_area1_hexes = list(delhi_area1['customer_location_hex_8'].unique())
delhi_area2 = pd.read_csv('/Users/rapido/Desktop/batching_demand/delhi/delhi_location_data_level1_demand_zone2.csv')
delhi_area2_hexes = list(delhi_area2['customer_location_hex_8'].unique())

In [20]:
#Create a list for all the hexes
hexes = [bangalore_area1_hexes, bangalore_area2_hexes, hyderabad_area1_hexes, hyderabad_area2_hexes, delhi_area1_hexes, delhi_area2_hexes]
group_name = ['bangalore_area1_hexes', 'bangalore_area2_hexes', 'hyderabad_area1_hexes', 'hyderabad_area2_hexes', 'delhi_area1_hexes', 'delhi_area2_hexes']

#Check the overlap in the hexes
overlap_dict = {}
for i in range(0, len(hexes)):
    hex_group = []
    for hex in hexes[i]:
        if hex in gp_enabled_hexes:
            hex_group.append(hex)
    overlap_dict[group_name[i]] = round(len(hex_group)/len(hexes[i]), 4)

In [28]:
#Create a df to check overlap by zone
overlap_df = pd.DataFrame(overlap_dict.items())
overlap_df.columns = ['area_name', 'overlap_%']
number_of_hexes = []
for x in hexes:
    number_of_hexes.append(len(x))
overlap_df['total_number_of_hexes'] = number_of_hexes
overlap_df['overlap_hexes'] = round(overlap_df['overlap_%']*overlap_df['total_number_of_hexes'], 0)
overlap_df

,area_name,overlap_%,total_number_of_hexes,overlap_hexes
0,bangalore_area1_hexes,0.5652,23,13.0
1,bangalore_area2_hexes,0.2500,16,4.0
2,hyderabad_area1_hexes,0.5312,32,17.0
3,hyderabad_area2_hexes,0.8750,16,14.0
4,delhi_area1_hexes,0.8293,41,34.0
5,delhi_area2_hexes,0.1111,18,2.0
